# Tag Products for E-commerce 

### Libraries & Keyword:

 - `Requests`
 - `Scrapy Selector`
 - `Css`
 - `Fuzzywuzzy`
 - `Pandas`

### Process
 - Step 1 - Scrape Web for up-to-date collection names
 - Step 2 - Find a string score for Fuzzywuzzy
 - Step 3 - Tag collection in a new column
 - Step 4 - Deal with NAs with domain knowledge 
 - Step 5 - Concat Na and not_Na dataset
 - Wrap up - What're the top 10 collections?


In [124]:
# load data and inspect

import pandas as pd 
df = pd.read_csv('kc_sales_sample.csv')
print(df.shape)
df.head()

(100, 6)


,product_title,variant_sku,variant_title,day,net_quantity,total_sales
0,Organic Necklace Petite,0110,Cavan Gold,2020-04-28,1,115.0
1,Villanova Wildcat Necklace,VU0110WILD,Cavan Gold,2020-04-25,1,115.0
2,Miami M Necklace,MOH0110M,Cavan Gold,2020-04-25,1,115.0
3,LIMITED EDITION MMXX Necklace,MMXX0110,Cavan Gold,2020-04-28,1,115.0
4,Princeton Tiger Necklace,PRI0110TIG,Cavan Gold,2020-04-22,1,115.0


## Step 1 - Scrape Web 

To get the most up-to-date information without the leg work of excel mannul input and typos

In [28]:
import requests
import scrapy 
from scrapy import Selector

# request data from web
url = 'https://kylecavan.com/pages/schools'
html = requests.get(url).content
sel = Selector(text = html)

# define css path
css = '.school-menu a::text'
collections = sel.css(css).extract()
print(type(collections), len(collections))

<class 'list'> 165


In [33]:
# create a collection dataframe
clt = pd.DataFrame(collections)
clt.rename(columns = {0:'Collection'}, inplace = True)
clt.head()

,Collection
0,Alabama
1,Auburn
2,Baylor
3,Berkeley
4,Boston College


## Step 2 - Decide a string score 

The `process.extract` returns the similar string in the iterable, a score for string similarity, and its postion in a tuple.

In [93]:
import fuzzywuzzy
from fuzzywuzzy import process

test_item = ['Villanova', 'Columbia', 'Alpha Omicron Pi']
test_score = [ process.extract(x, df['product_title'], limit = 3) for x in test_item]
test_score

[[('Villanova Wildcat Necklace', 90, 1),
  ('Villanova V Necklace', 90, 15),
  ('Villanova Organic Necklace', 90, 19)],
 [('Columbia Crown Necklace', 90, 55),
  ('Columbia C Necklace', 90, 56),
  ('Columbia Low Library Necklace', 90, 65)],
 [('Alpha Omicron Pi Rose Necklace Petite', 90, 71),
  ('Alpha Omicron Pi Rose Necklace Petite', 90, 89),
  ('Alpha Delta Pi Letters Necklace Petite', 86, 8)]]

### Result:

__90__ is a good score, enough to filter out nuance.   
Example, `Alpha Omicron Pi` = 90, while `Alpha Delta Pi` = 86.

## Step 3 - Tag collection in a new column

In [84]:

for item in clt['Collection']:

    matches = process.extract(item, df['product_title'], limit = df.shape[0])  
    # compare all titles with collection strings
    
    for potential_match in matches:
   
        if potential_match[1]>=90:  # the score discovered above
            match_test = df['product_title'] == potential_match[0]  # find the match in df.product_title
            df.loc[match_test, 'Collection'] = item  # tag it
    

In [94]:
df.head()

,product_title,variant_sku,variant_title,day,net_quantity,total_sales,Collection
0,Organic Necklace Petite,0110,Cavan Gold,2020-04-28,1,115.0,NaN
1,Villanova Wildcat Necklace,VU0110WILD,Cavan Gold,2020-04-25,1,115.0,Villanova
2,Miami M Necklace,MOH0110M,Cavan Gold,2020-04-25,1,115.0,Miami
3,LIMITED EDITION MMXX Necklace,MMXX0110,Cavan Gold,2020-04-28,1,115.0,NaN
4,Princeton Tiger Necklace,PRI0110TIG,Cavan Gold,2020-04-22,1,115.0,Princeton


## Step 4 - Deal with NAs

In [95]:
na_flt = df['Collection'].isna()
not_na = df[~na_flt]
na = df[na_flt]
na.shape

(39, 7)

In [96]:
na.head()

,product_title,variant_sku,variant_title,day,net_quantity,total_sales,Collection
0,Organic Necklace Petite,0110,Cavan Gold,2020-04-28,1,115.0,NaN
3,LIMITED EDITION MMXX Necklace,MMXX0110,Cavan Gold,2020-04-28,1,115.0,NaN
12,Miss America Necklace Petite,MAO0110,Cavan Gold,2020-04-20,1,115.0,NaN
14,TCU Frog Fountain Necklace,TCU0110FOUNTAIN,Cavan Gold,2020-04-16,1,115.0,NaN
16,Organic Necklace,0112,Sterling Silver,2020-04-21,1,113.4,NaN


## Stop and think:

NA values always occur in real business.   
Here, based on domain knowlege and company website, __MMXX__  and __Miss America__ are two other collections ~~not in school list~~.  
Other than these two, the rest are __custom__ pieces not for online sale.

## Fill Na

In [102]:
# make filters
mx = na['product_title'].str.contains('MMXX')
ma = na['product_title'].str.contains('Miss America')

# fill na
na.loc[mx, 'Collection'] = 'MMXX'
na.loc[ma, 'Collection'] = 'Miss America'
na.loc[~(mx + ma), 'Collection'] = 'Custom'

na.head()

,product_title,variant_sku,variant_title,day,net_quantity,total_sales,Collection
0,Organic Necklace Petite,0110,Cavan Gold,2020-04-28,1,115.0,Custom
3,LIMITED EDITION MMXX Necklace,MMXX0110,Cavan Gold,2020-04-28,1,115.0,MMXX
12,Miss America Necklace Petite,MAO0110,Cavan Gold,2020-04-20,1,115.0,Miss America
14,TCU Frog Fountain Necklace,TCU0110FOUNTAIN,Cavan Gold,2020-04-16,1,115.0,Custom
16,Organic Necklace,0112,Sterling Silver,2020-04-21,1,113.4,Custom


In [103]:
na['Collection'].isna().sum()

0

## Step 5 - Concat na dataset to not_na dataset

In [123]:
df_filled = pd.concat([not_na, na])
df_filled.reset_index(drop = True, inplace = True)

df_filled.head(3)

,product_title,variant_sku,variant_title,day,net_quantity,total_sales,Collection
0,Villanova Wildcat Necklace,VU0110WILD,Cavan Gold,2020-04-25,1,115.0,Villanova
1,Miami M Necklace,MOH0110M,Cavan Gold,2020-04-25,1,115.0,Miami
2,Princeton Tiger Necklace,PRI0110TIG,Cavan Gold,2020-04-22,1,115.0,Princeton


In [122]:
df_filled.tail(3)

,product_title,variant_sku,variant_title,day,net_quantity,total_sales,Collection
97,LIMITED EDITION MMXX Necklace,MMXX0109,Sterling Silver,2020-04-07,1,85.0,MMXX
98,LIMITED EDITION MMXX Necklace,MMXX0109,Sterling Silver,2020-04-12,1,82.9,MMXX
99,St. Lawrence Organic Necklace Petite,SLU0110,Cavan Gold,2020-04-01,1,80.5,Custom


In [110]:
df_filled.shape  # same shape as before

(100, 7)

## Wrapping up 
### What're the __top 10__ collections?

In [121]:
df_filled.groupby('Collection').agg({'net_quantity':'sum'}).sort_values('net_quantity', ascending = False).head(10)

,net_quantity
Collection,
Custom,18
MMXX,17
Villanova,9
Virginia Tech,5
Miss America,4
Miami,4
Alpha Delta Pi,3
TCU,3
Purdue,3


## Well Done!